<a href="https://colab.research.google.com/github/richapatel93/Data-Mining-Class/blob/main/NLP_Disaster_Tweets_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import files

uploaded = files.upload()


Saving sample_submission.csv to sample_submission (1).csv


In [9]:
from google.colab import files

uploaded = files.upload()


Saving test.csv to test (1).csv


In [10]:
from google.colab import files

uploaded = files.upload()


Saving train.csv to train (1).csv


In [11]:
import os
print(os.listdir())


['.config', 'train (1).csv', 'sample_submission.csv', 'train.csv', 'test.csv', 'sample_submission (1).csv', 'test (1).csv', 'sample_data']


In [12]:
#load the data:
import pandas as pd

train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

# Preview the data
print(train_data.head())
print(test_data.head())
print(sample_submission.head())


   id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1  
   id keyword location                                               text
0   0     NaN      NaN                 Just happened a terrible car crash
1   2     NaN      NaN  Heard about #earthquake is different cities, s...
2   3     NaN      NaN  there is a forest fire at spot pond, geese are...
3   9     NaN      NaN           Apocalypse lighting. #Spokane #wildfires
4  11     NaN      NaN      Typhoon Soudelor kills 28 in China and Taiwan
   id  target
0   0  

In [13]:
#Verify Data Dimensions
print(f"Train data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")
print(f"Sample submission shape: {sample_submission.shape}")


Train data shape: (7613, 5)
Test data shape: (3263, 4)
Sample submission shape: (3263, 2)


In [14]:
#inspect missing values:
print(train_data.isnull().sum())
print(test_data.isnull().sum())


id             0
keyword       61
location    2533
text           0
target         0
dtype: int64
id             0
keyword       26
location    1105
text           0
dtype: int64


In [15]:
# Fill missing values in 'keyword' and 'location' with "none"
train_data['keyword'] = train_data['keyword'].fillna("none")
train_data['location'] = train_data['location'].fillna("none")

test_data['keyword'] = test_data['keyword'].fillna("none")
test_data['location'] = test_data['location'].fillna("none")

# Check for missing values in the 'text' column
print(f"Missing values in 'text': {train_data['text'].isnull().sum()}")


Missing values in 'text': 0


In [17]:
# Encode Target Column
print(train_data['target'].value_counts())


target
0    4342
1    3271
Name: count, dtype: int64


In [16]:
#Text Preprocessing
import re
import nltk
from nltk.corpus import stopwords

# Download stopwords (if not already downloaded)
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    # Remove mentions and hashtags
    text = re.sub(r"@\w+|#\w+", "", text)
    # Remove special characters and numbers
    text = re.sub(r"[^A-Za-z\s]", "", text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

# Apply preprocessing to the 'text' column
train_data['text'] = train_data['text'].apply(preprocess_text)
test_data['text'] = test_data['text'].apply(preprocess_text)

# Preview the cleaned text
print(train_data['text'].head())


0                    deeds reason may allah forgive us
1                forest fire near la ronge sask canada
2    residents asked shelter place notified officer...
3          people receive evacuation orders california
4               got sent photo ruby smoke pours school
Name: text, dtype: object


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
#Split the Training Data

from sklearn.model_selection import train_test_split

train, val = train_test_split(train_data, test_size=0.2, random_state=42)

print(f"Training set size: {train.shape}")
print(f"Validation set size: {val.shape}")


Training set size: (6090, 5)
Validation set size: (1523, 5)


# Tokenize and Prepare for Modeling

In [20]:
!pip install --upgrade keras-nlp tensorflow


In [21]:
!pip install transformers


In [22]:
#Load the DistilBERT model and tokenizer:

from transformers import DistilBertTokenizer, TFDistilBertModel

# Load the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = TFDistilBertModel.from_pretrained("distilbert-base-uncased")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [25]:
# Split the training data
from sklearn.model_selection import train_test_split
train, val = train_test_split(train_data, test_size=0.2, random_state=42)

# Tokenize the datasets
def tokenize_texts(texts):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors="tf",
        max_length=128,  # Adjust as needed
    )

# Tokenize datasets
train_encodings = tokenize_texts(train['text'].tolist())
val_encodings = tokenize_texts(val['text'].tolist())
test_encodings = tokenize_texts(test_data['text'].tolist())

# Convert labels to tensors
train_labels = tf.convert_to_tensor(train['target'].values)
val_labels = tf.convert_to_tensor(val['target'].values)

# Print the shapes of the tokenized datasets and labels
print(f"Train encodings shape: {train_encodings['input_ids'].shape}")
print(f"Validation encodings shape: {val_encodings['input_ids'].shape}")
print(f"Test encodings shape: {test_encodings['input_ids'].shape}")
print(f"Train labels shape: {train_labels.shape}")
print(f"Validation labels shape: {val_labels.shape}")


Train encodings shape: (6090, 39)
Validation encodings shape: (1523, 40)
Test encodings shape: (3263, 38)
Train labels shape: (6090,)
Validation labels shape: (1523,)
